객체 탐지 (Object Detection)
   - 한 이미지에서 객체와 그 경계 상자(bounding box)를 탐지
   - 객체 탐지 알고리즘은 일반적인 이미지를 입력으로 받고, 경계 상자와 객체 클래스 리스트를 출력
   - 경계 상자에 대해 그에 대응하는 예측 클래스와 클래스의 신뢰도(confidence)를 출력

   - Applications
      - 자율 주행 자동차에서 다른 자동차와 보행자를 찾을 때
      - 의료 분야에서 방사선 사진을 사용해 종양이나 위험한 조직을 찾을 때
      - 제조업에서 조립 로봇이 제품을 조립하거나 수리할 때
      - 보안 산업에서 위협을 탐지하거나 사람을 셀 때

1. Bounding Box : 이미지에서 하나의 객체 전체를 포함하는 가장 작은 직사각형 

2. IOU(Intersection Over Union) : 실측값(Ground Truth)과 모델이 예측한 값이 얼마나 겹치는지를 나타내는 지표

In [1]:
# IOU 함수로 구현
import numpy as np

def compute_iou(pred_box, gt_box):
    x1 = np.maximum(pred_box[0], gt_box[0])
    y1 = np.maximum(pred_box[1], gt_box[1])
    x2 = np.maximum(pred_box[2], gt_box[2])
    y2 = np.maximum(pred_box[3], gt_box[3])
    
    intersection = np.maximum(x2 - x1, 0) * np.maximum(y2- y1, 0)
    
    pred_box_area = (pred_box[2] - pred_box[0]) * (pred_box[3] - pred_box[1])
    gt_box_area = (gt_box[2] - gt_box[0]) * (gt_box[3] - gt_box[1])
    
    union = pred_box_area + gt_box_area - intersection
    
    iou = intersection / union
    
    return iou

3. NMS(Non-Maximum Suppression, 비최댓값 억제)
   - 확률이 가장 높은 상자와 겹치는 상자들을 제거하는 과정
   - 최댓값을 갖지 않는 상자들을 제거
   - 과정
     1. 확률 기준으로 모든 상자를 정렬하고 먼저 가장 확률이 높은 상자를 취함
     2. 각 상자에 대해 다른 모든 상자와의 IOU를 계산
     3. 특정 임곗값을 넘는 상자는 제거

In [6]:
import numpy as np

def non_max_suppression_fast(boxes, overlap_thresh):
    if len(boxes) == 0:
        return []
    
    if boxes.dtype.kind == 'i':
        boxes = boxes.astype("float")
        
    pick = []
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
    
    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.maximum(x2[i], x2[idxs[:last]])
        yy2 = np.maximum(y2[i], y2[idxs[:last]])
        
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        
        overlap = (w * h) / area[idxs[:last]]
        
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlap_thresh)[0])))
    
    return boxes[pick].astype('int')

모델 성능 평가

1. 정밀도와 재현율(Precision and Recall)
   - 정밀도-재현율 곡선(Precision-recall curve)
      - 신뢰도 임곗값마다 모델의 정밀도와 재현율 시각화
      - 모든 bounding box와 함께 모델이 예측의 장확성을 얼마나 확실하는지 0~1 사이의 숫자로 나타내는 신뢰도
      - 임계값 T에 따라 정밀도와 재현율이 달라짐
      - ex) 모델이 보행자를 탐지하고 있으면 특별한 이유없이 차를 세우더라도 어떤 보행자도 놓치지 않도록 재현율을 높여야함
      모델이 투자 기회를 탐지하고 있다면 일부 기회를 놓치게 되더라도 잘못된 기회에 돈을 거는 일을 피하기 위해 정밀도를 높여야함

2. AP(Average Precision, 평균 정밀도)와 mAP(mean Average Precision)

YOLO(You Only Look Once)
   - 가장 빠른 객체 검출 알고리즘 중 하나
   - 256x256 사이즈의 이미지
   - 자세한 내용 : https://www.kaggle.com/aruchomu/yolo-v3-object-detection-in-tensorflow

YOLO Backbone
   - 백본 모델(backbone model) 기반
   - 특징 추출기(Feature Extractor)라고도 불림
   - YOLO는 자체 맞춤 아키텍쳐 사용
   - 어떤 특징 추출기 아키텍쳐를 사용했는지 따라 성능이 달라짐

앵커 박스(Anchor Box)
  - YOLOv2에서 도입
  - 사전 정의된 상자(prior box)
  - 객체에 가장 근접한 앵커 박스를 맞추고 신경망을 사용해 앵커 박스의 크기를 조정하는 과정

YOLOv3 설치하기
   - 1. git 설치하기 : https://ghostweb.tistory.com/888
   - 2. wget 설치하기 : https://synchronized.tistory.com/entry/wget%EC%9D%84-%EC%9C%88%EB%8F%84%EC%9A%B0%EC%97%90%EC%84%9C-%EC%82%AC%EC%9A%A9%ED%95%B4%EB%B3%B4%EC%9E%90
   - 3. yolov3 설치하기 : https://deepflowest.tistory.com/369
   

In [1]:
import sys
from absl import app, logging, flags
from absl.flags import FLAGS
import time
import cv2
import numpy as np
import tensorflow as tf
from yolov3_tf2.models import YoloV3, YoloV3Tiny
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs

In [16]:
# flags.DEFINE_string('classes', './data/coco.names', 'path to classes file')
# flags.DEFINE_string('weights', './checkpoints/yolov3.tf', 'path to weights file')
# flags.DEFINE_boolean('tiny', False, 'yolov3 or yolov3-tiny')
# flags.DEFINE_integer('size', 416, 'resize images to')
# flags.DEFINE_string('image', './data/firl.png', 'path to input image')
# flags.DEFINE_string('tfrecord', None, 'tfrecord instead of image')
# flags.DEFINE_string('output', './output.jpg', 'path to output image')
# flags.DEFINE_integer('num_classes', 80, 'number of classes in the model')

# app._run_init(['yolov3'], app.parse_flags_with_usage)

# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

DuplicateFlagError: The flag 'weights' is defined twice. First from C:\Users\bitcamp\.conda\envs\yolov3-tf2-gpu\lib\site-packages\ipykernel_launcher.py, Second from C:\Users\bitcamp\.conda\envs\yolov3-tf2-gpu\lib\site-packages\ipykernel_launcher.py.  Description from first occurrence: path to weights file

In [9]:
FLAGS.image = 'data/meme.jpg'

In [15]:
if FLAGS.tiny:
    yolo = YoloV3Tiny(classes=FLAGS.num_classes)
else:
    yolo = YoloV3(classes=FLAGS.num_classes)

yolo.load_weights(FLAGS.weights).expect_partial()
logging.info('weights loaded')

class_names = [c.strip() for c in open(FLAGS.classes).readlines()]
logging.info('classes loaded')

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for ./checkpoints/yolov3.tf

In [ ]:
img_raw = tf.image.decode_image(open(FLAGS.image, 'rb').read(), channels=3)

img = tf.expand_dims(img_raw, 0)
img = transform_images(img, FLAGS.size)

t1 = time.time()
boxes, scores, classes, nums = yolo(img)
t2 = time.time()
logging.info('time: {}'.format(t2 - t1))

logging.info('detections:')
for i in range(nums[0]):
    logging.info('  {}, {}, {}'.format(class_names[int(classes[0][i])],
                                       np.array(scores[0][i]),
                                       np.array(boxes[0][i])))
img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)
img = draw_outputs(img, (boxes, scores, classes, nums), class_names)

In [12]:
from IPython.display import Image, display

display(Image(data=bytes(cv2.imencode('.jpg', img)[1]), width=800))

NameError: name 'img' is not defined